In [1]:
import pandas as pd
occu_df = pd.read_csv('datatraining.txt', sep = ',' )
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline
occu_df_1 = occu_df[occu_df.Occupancy !=0]
occu_df_0 = occu_df[occu_df.Occupancy !=1]

In [5]:
occu_df_1.shape,occu_df_0.shape

((1729, 7), (6414, 7))

In [2]:
import random
def shuffle(df):
    index = list(df.index)
    random.shuffle(index)
    df = df.ix[index]
    df.reset_index()
    return df

In [3]:
occu_df_0_s = shuffle(occu_df_0)
occu_df_00 = occu_df_0_s[0:1729]
train_df_0 = pd.concat([occu_df_00 , occu_df_1])

/Users/inagakiyuuya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [4]:
train_df_0['CO2_n'] = (train_df_0['CO2'] - train_df_0['CO2'].mean()) / train_df_0['CO2'].std()
train_df_0['HumidityRatio_n'] = (train_df_0['HumidityRatio'] - train_df_0['HumidityRatio'].mean()) / train_df_0['HumidityRatio'].std()train_df_0['Temperature_n'] = (train_df_0['Temperature'] - train_df_0['Temperature'].mean()) / train_df_0['Temperature'].std()
train_df_0['Humidity_n'] = (train_df_0['Humidity'] - train_df_0['Humidity'].mean()) / train_df_0['Humidity'].std()
train_df_0['Light_n'] = (train_df_0['Light'] - train_df_0['Light'].mean()) / train_df_0['Light'].std()

In [5]:
train_df_0.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Temperature_n,Humidity_n,Light_n,CO2_n,HumidityRatio_n
1457,2015-02-05 18:07:00,22.00,24.10,0.0,852.0,0.003937,0,0.981697,-0.372024,-1.071498,0.216144,-0.110205
4538,2015-02-07 21:28:00,19.89,26.79,0.0,437.0,0.003844,0,-1.073144,0.092730,-1.071498,-0.824294,-0.209373
394,2015-02-05 00:23:59,21.29,25.20,0.0,454.0,0.003942,0,0.290258,-0.181976,-1.071498,-0.781674,-0.105146
7447,2015-02-09 21:56:59,20.50,33.50,0.0,632.0,0.005000,0,-0.479090,1.252021,-1.071498,-0.335414,1.014745
7497,2015-02-09 22:46:59,20.50,33.09,0.0,562.0,0.004939,0,-0.479090,1.181185,-1.071498,-0.510909,0.949467


In [6]:
train_df = train_df_0.drop(['date' , 'Temperature' , 'Humidity' , 'Light' , 'CO2' , 'HumidityRatio'] , axis=1)

In [9]:
train_df.tail()

,Occupancy,Temperature_n,Humidity_n,Light_n,CO2_n,HumidityRatio_n
8139,1,0.056532,1.700793,0.826400,0.053811,1.626900
8140,1,0.056532,1.683084,0.826400,0.059452,1.609986
8141,1,0.105225,1.700361,0.826400,0.082015,1.644812
8142,1,0.105225,1.728869,0.826400,0.136753,1.672126
8143,1,0.105225,1.718502,0.887764,0.138425,1.662194


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [11]:
#目的変数を作る
Yt = train_df.Occupancy
Y_train = np.ravel(Yt)

In [13]:
#説明変数を作る
X_train = train_df.drop([ 'Occupancy'] , axis=1)

In [14]:
log_model = LogisticRegression()

In [15]:
log_model.fit(X_train , Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
coeff_df = pd.DataFrame([X_train.columns , log_model.coef_[0]]).T
coeff_df

,0,1
0,Temperature_n,-1.10717
1,Humidity_n,-0.0748367
2,Light_n,4.69001
3,CO2_n,2.05274
4,HumidityRatio_n,0.0879494


In [18]:
test1_df = pd.read_csv('datatest.txt')
test2_df = pd.read_csv('datatest2.txt')

In [19]:
Y1 = test1_df.Occupancy
Y_test1 = np.ravel(Y1)
Y2 = test2_df.Occupancy
Y_test2 = np.ravel(Y2)

In [20]:
X_test1 = test1_df.drop(['date' , 'Occupancy'] , axis=1)
X_test2 = test2_df.drop(['date' , 'Occupancy'] , axis=1)

In [21]:
pred_test1 = log_model.predict(X_test1)
pred_test2 = log_model.predict(X_test2)

In [23]:
df = pd.DataFrame(Y_test1 , columns=['data'])

In [24]:
df['est'] = pd.DataFrame(pred_test1)

In [25]:
def gap(t):
    if t == 0:
        return 0
    else:
        return 1

In [26]:
df['dif'] =Y_test1-pred_test1

In [27]:
df['gap'] =df['dif'].apply(gap)

In [28]:
df

,data,est,dif,gap
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0
5,1,1,0,0
6,1,1,0,0
7,1,1,0,0
8,1,1,0,0
9,1,1,0,0


In [29]:
df.sum()

data     972
est     2665
dif    -1693
gap     1693
dtype: int64

In [30]:
test1_df['Temperature_n'] = (test1_df['Temperature'] - test1_df['Temperature'].mean()) / test1_df['Temperature'].std()
test1_df['Humidity_n'] = (test1_df['Humidity'] - test1_df['Humidity'].mean()) / test1_df['Humidity'].std()
test1_df['Light_n'] = (test1_df['Light'] - test1_df['Light'].mean()) / test1_df['Light'].std()
test1_df['CO2_n'] = (test1_df['CO2'] - test1_df['CO2'].mean()) / test1_df['CO2'].std()
test1_df['HumidityRatio_n'] = (test1_df['HumidityRatio'] - test1_df['HumidityRatio'].mean()) / test1_df['HumidityRatio'].std()

In [31]:
test1_df.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy,Temperature_n,Humidity_n,Light_n,CO2_n,HumidityRatio_n
140,2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1,2.204349,0.376743,1.566568,0.106920,1.207314
141,2015-02-02 14:19:59,23.7180,26.290,578.400000,760.400000,0.004773,1,2.221858,0.384130,1.539391,0.145187,1.221232
142,2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1,2.233531,0.359508,1.516477,0.176848,1.208934
143,2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1,2.226235,0.316419,1.201077,0.194216,1.173919
144,2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1,2.256877,0.347197,1.180494,0.208737,1.211295


In [32]:
test1_df_n = test1_df.drop(['date' , 'Temperature' , 'Humidity' , 'Light' , 'CO2' , 'HumidityRatio'] , axis=1)

In [33]:
test2_df['Temperature_n'] = (test2_df['Temperature'] - test2_df['Temperature'].mean()) / test2_df['Temperature'].std()
test2_df['Humidity_n'] = (test2_df['Humidity'] - test2_df['Humidity'].mean()) / test2_df['Humidity'].std()
test2_df['Light_n'] = (test2_df['Light'] - test2_df['Light'].mean()) / test2_df['Light'].std()
test2_df['CO2_n'] = (test2_df['CO2'] - test2_df['CO2'].mean()) / test2_df['CO2'].std()
test2_df['HumidityRatio_n'] = (test2_df['HumidityRatio'] - test2_df['HumidityRatio'].mean()) / test2_df['HumidityRatio'].std()

In [34]:
test2_df_n = test2_df.drop(['date' , 'Temperature' , 'Humidity' , 'Light' , 'CO2' , 'HumidityRatio'] , axis=1)

In [35]:
Y1 = test1_df_n.Occupancy
Y_test1_n = np.ravel(Y1)
Y2 = test2_df_n.Occupancy
Y_test2_n = np.ravel(Y2)

In [36]:
X_test1_n = test1_df_n.drop(['Occupancy'] , axis=1)
X_test2_n = test2_df_n.drop(['Occupancy'] , axis=1)

In [37]:
pred_test1_n = log_model.predict(X_test1_n)
pred_test2_n = log_model.predict(X_test2_n)

In [38]:
df_n = pd.DataFrame(Y_test1_n , columns=['data'])

In [39]:
df_n['est'] = pd.DataFrame(pred_test1_n)

In [40]:
df_n['dif'] =Y_test1_n - pred_test1_n

In [41]:
df_n['gap'] =df_n['dif'].apply(gap)

In [42]:
df_n.sum()

data     972
est     1027
dif      -55
gap       57
dtype: int64